<a href="https://colab.research.google.com/github/gustavohn578/Alura_IA_2aEdicao/blob/main/Analise_Curriculos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Análise de currículos</h1>
É usual que as ferramentas de pesquisa de currículos usem uma série de filtros e palavras chaves para buscar resultados.</br>
São ferramentas úteis, mas ainda assim exigem muito tempo na seleção e análise de candidatos.</br>
O objetivo deste projeto é executar de forma automática a avaliação de uma base de currículos para definir qual deles melhor atende uma lista de requisitos.<br>



<h2>Instalar o SDK</h2>

In [1]:
!pip install -U -q google-generativeai

<h2>Importar bibliotecas e configurar a API Key</h2>

In [2]:
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

my_api_key = userdata.get('MY_KEY')

genai.configure(api_key=my_api_key)

<h2>Listar modelos</h2>

In [3]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


<h2>Gerar base de currículos para simulação</h2>
Em uma aplicação real existirá uma base de currículos previamente selecionados ou coletados.<br>
Para demonstração usaremos aqui uma pequena base de currículos simulados .<br>
O Google Ai Studio foi usado para criar o prompt e gerar os currículos que depois foram copiados.</br><br>
<h3>Comentário</h3>
A abordagem para gerar os currículos simulados via código pela própria API apresentou dificuldades inesperadas com timeout de excecução.</br> Após aumentar   o timeout no parâmetro <i>request_options</i> do método <i>model.generate_content</i> algumas vezes sem sucesso, decidiu-se por copiar o resultado do prompt.

In [4]:
CURRICULO1 = {
    "Candidato": "Candidato 1",
    "Curriculo": {
        "cargo": "Desenvolvedor Full Stack",
        "resumo": "Desenvolvedor Full Stack com +5 anos de experiência na construção e manutenção de aplicações web complexas, com foco em performance, escalabilidade e usabilidade. Habilidade comprovada em trabalhar com times ágeis, liderando projetos e entregando soluções inovadoras.",
        "habilidades": "JavaScript, Python, Java, C#, React, Node.js, Angular, Django, Spring Boot, SQL Server, PostgreSQL, MongoDB, AWS, Azure, GCP, Git, Github, Scrum, Kanban",
        "experiencia profissional": "Desenvolvedor Full Stack | Empresa X | [Cidade, Estado] | [Data Inicial] - [Data Final]\nDesenvolvedor Front-End | Empresa Y | [Cidade, Estado] | [Data Inicial] - [Data Final]",
        "formacao": "Bacharelado em Ciência da Computação | Universidade Z | [Ano de Conclusão]",
        "certificações": "",
        "projetos": "Desenvolvimento de um aplicativo mobile para [descrição do projeto], disponível na App Store e Google Play.\nCriação de um blog pessoal sobre desenvolvimento web, compartilhando conhecimento e experiências com a comunidade.",
        "idiomas": "Português (Nativo)\nInglês (Fluente)"
    }
}

CURRICULO2 = {
    "Candidato": "Candidato 2",
    "Curriculo": {
        "cargo": "Cientista de Dados",
        "resumo": "Cientista de Dados com +3 anos de experiência em análise e modelagem de dados, com foco em Machine Learning e Inteligência Artificial. Habilidade comprovada em transformar dados em insights estratégicos para otimizar processos e auxiliar na tomada de decisões.",
        "habilidades": "Python, R, Pandas, NumPy, Scikit-learn, TensorFlow, Keras, Machine Learning, Regressão, Classificação, Clusterização, Deep Learning, Tableau, Power BI, SQL Server, MySQL, Hadoop, Spark",
        "experiencia profissional": "Cientista de Dados | Empresa A | [Cidade, Estado] | [Data Inicial] - [Data Final]\nAnalista de Dados | Empresa B | [Cidade, Estado] | [Data Inicial] - [Data Final]",
        "formacao": "Mestrado em Estatística | Universidade W | [Ano de Conclusão]\nBacharelado em Matemática | Universidade X | [Ano de Conclusão]",
        "certificações": "AWS Certified Machine Learning - Specialty",
        "projetos": "Desenvolvimento de um projeto de machine learning para [descrição do projeto], disponível no Github.",
        "idiomas": "Português (Nativo)\nInglês (Avançado)"
    }
}

CURRICULO3 = {
    "Candidato": "Candidato 3",
    "Curriculo": {
        "cargo": "Engenheiro de DevOps",
        "resumo": "Engenheiro de DevOps com +4 anos de experiência em automatização de infraestrutura, integração contínua e entrega contínua (CI/CD). Habilidade comprovada em otimizar o processo de desenvolvimento de software, aumentar a eficiência da equipe e garantir a qualidade do produto final.",
        "habilidades": "AWS, Azure, GCP, Docker, Kubernetes, Ansible, Puppet, Chef, CI/CD, Jenkins, GitLab CI, CircleCI, Linux, Windows Server, Bash, Python, Prometheus, Grafana",
        "experiencia profissional": "Engenheiro de DevOps | Empresa C | [Cidade, Estado] | [Data Inicial] - [Data Final]\nAdministrador de Sistemas | Empresa D | [Cidade, Estado] | [Data Inicial] - [Data Final]",
        "formacao": "Bacharelado em Engenharia da Computação | Universidade Y | [Ano de Conclusão]",
        "certificações": "AWS Certified Solutions Architect - Associate\nCertified Kubernetes Administrator (CKA)",
        "projetos": "Criação de um ambiente de desenvolvimento automatizado utilizando Docker e Ansible.",
        "idiomas": "Português (Nativo)\nInglês (Intermediário)"
    }
}

CURRICULO4 = {
    "Candidato": "Candidato 4",
    "Curriculo": {
        "cargo": "Analista de Segurança da Informação",
        "resumo": "Analista de Segurança da Informação com +2 anos de experiência em proteção de dados, prevenção de ameaças e resposta a incidentes. Habilidade comprovada em identificar vulnerabilidades, implementar medidas de segurança e garantir a conformidade com as melhores práticas.",
        "habilidades": "Firewall, VPN, Intrusion Detection System (IDS), Intrusion Prevention System (IPS), OWASP Top 10, testes de penetração, análise de código estático, Criptografia, controle de acesso, backup e recuperação de dados, Investigação de incidentes, análise forense, plano de recuperação de desastres, ISO 27001, GDPR, LGPD",
        "experiencia profissional": "Analista de Segurança da Informação | Empresa E | [Cidade, Estado] | [Data Inicial] - [Data Final]\nEstagiário de Segurança da Informação | Empresa F | [Cidade, Estado] | [Data Inicial] - [Data Final]",
        "formacao": "Bacharelado em Segurança da Informação | Universidade Z | [Ano de Conclusão]",
        "certificações": "Certified Information Systems Security Professional (CISSP)\nCompTIA Security+",
        "projetos": "Configuração de um laboratório de segurança da informação para realizar testes de penetração.",
        "idiomas": "Português (Nativo)\nInglês (Técnico)"
    }
}

CURRICULO5 = {
    "Candidato": "Candidato 5",
    "Curriculo": {
        "cargo": "Gerente de Projetos de TI",
        "resumo": "Gerente de Projetos de TI com +7 anos de experiência em planejamento, execução e entrega de projetos de tecnologia. Habilidade comprovada em liderar equipes, gerenciar orçamentos, mitigar riscos e garantir a satisfação do cliente.",
        "habilidades": "Waterfall, Agile (Scrum, Kanban), Jira, Microsoft Project, Asana, Gerenciamento de Riscos e Problemas, Comunicação Eficaz, Liderança e Motivação de Equipes, Negociação e Gestão de Conflitos",
        "experiencia profissional": "Gerente de Projetos de TI | Empresa G | [Cidade, Estado] | [Data Inicial] - [Data Final]\nCoordenador de Projetos de TI | Empresa H | [Cidade, Estado] | [Data Inicial] - [Data Final]",
        "formacao": "MBA em Gerenciamento de Projetos | Universidade W | [Ano de Conclusão]\nBacharelado em Ciência da Computação | Universidade X | [Ano de Conclusão]",
        "certificações": "Project Management Professional (PMP)\nCertified Scrum Master (CSM)",
        "projetos": "Voluntariado em projetos de tecnologia para organizações sem fins lucrativos.",
        "idiomas": "Português (Nativo)\nInglês (Fluente)\nEspanhol (Básico)"
    }
}

curriculos = [CURRICULO1, CURRICULO2, CURRICULO3, CURRICULO4, CURRICULO5]



In [5]:
for m in curriculos:
  print(m["Candidato"])

Candidato 1
Candidato 2
Candidato 3
Candidato 4
Candidato 5


<h2>Definir a função currículo para texto</h2>
Definir uma função para **converter** a estrutura do currículo para uma única sttring que possa ser usada para gerar embeddings.

In [6]:
def curriculo_para_texto(curriculo):
  """Converte um currículo em um único texto."""
  texto = f"Cargo: {curriculo['Curriculo']['cargo']}\n"
  texto += f"Resumo: {curriculo['Curriculo']['resumo']}\n"
  texto += f"Habilidades: {curriculo['Curriculo']['habilidades']}\n"
  texto += f"Experiência Profissional: {curriculo['Curriculo']['experiencia profissional']}\n"
  texto += f"Formação: {curriculo['Curriculo']['formacao']}\n"
  texto += f"Certificações: {curriculo['Curriculo']['certificações']}\n"
  texto += f"Projetos: {curriculo['Curriculo']['projetos']}\n"
  texto += f"Idiomas: {curriculo['Curriculo']['idiomas']}"
  return texto

#textos_curriculos = [curriculo_para_texto(c) for c in curriculos]

for m in curriculos:
  m["Curriculo"] = curriculo_para_texto(m)

curriculos

[{'Candidato': 'Candidato 1',
  'Curriculo': 'Cargo: Desenvolvedor Full Stack\nResumo: Desenvolvedor Full Stack com +5 anos de experiência na construção e manutenção de aplicações web complexas, com foco em performance, escalabilidade e usabilidade. Habilidade comprovada em trabalhar com times ágeis, liderando projetos e entregando soluções inovadoras.\nHabilidades: JavaScript, Python, Java, C#, React, Node.js, Angular, Django, Spring Boot, SQL Server, PostgreSQL, MongoDB, AWS, Azure, GCP, Git, Github, Scrum, Kanban\nExperiência Profissional: Desenvolvedor Full Stack | Empresa X | [Cidade, Estado] | [Data Inicial] - [Data Final]\nDesenvolvedor Front-End | Empresa Y | [Cidade, Estado] | [Data Inicial] - [Data Final]\nFormação: Bacharelado em Ciência da Computação | Universidade Z | [Ano de Conclusão]\nCertificações: \nProjetos: Desenvolvimento de um aplicativo mobile para [descrição do projeto], disponível na App Store e Google Play.\nCriação de um blog pessoal sobre desenvolvimento web

<h2> Preencher o dataframe</h2>

In [7]:
df = pd.DataFrame(curriculos)
df

,Candidato,Curriculo
0,Candidato 1,Cargo: Desenvolvedor Full Stack\nResumo: Desen...
1,Candidato 2,Cargo: Cientista de Dados\nResumo: Cientista d...
2,Candidato 3,Cargo: Engenheiro de DevOps\nResumo: Engenheir...
3,Candidato 4,Cargo: Analista de Segurança da Informação\nRe...
4,Candidato 5,Cargo: Gerente de Projetos de TI\nResumo: Gere...


In [8]:
model = "models/embedding-001"

<h2>Adicionar embeddings de cada currículo ao dataframe</h2>

In [9]:
def embed_fn(candidato, curriculo):
  return genai.embed_content(model=model,
                                 content=curriculo,
                                 title=candidato,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [10]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Candidato"], row["Curriculo"]), axis=1)
df

,Candidato,Curriculo,Embeddings
0,Candidato 1,Cargo: Desenvolvedor Full Stack\nResumo: Desen...,"[0.023134675, -0.056388088, -0.026959823, 0.00..."
1,Candidato 2,Cargo: Cientista de Dados\nResumo: Cientista d...,"[-0.006379591, -0.07124528, -0.017808588, 0.02..."
2,Candidato 3,Cargo: Engenheiro de DevOps\nResumo: Engenheir...,"[0.031235173, -0.05558979, -0.026302753, 0.007..."
3,Candidato 4,Cargo: Analista de Segurança da Informação\nRe...,"[0.025696216, -0.05785462, -0.03956894, 0.0135..."
4,Candidato 5,Cargo: Gerente de Projetos de TI\nResumo: Gere...,"[-0.004269714, -0.03849035, -0.020453298, 0.01..."


<h2>Definir a função de busca de candidato</h2>
A função receberá um texto de consulta que contém a descrição de uma vaga.

In [11]:
def buscar_melhor_candidato(consulta, base, model):
  embedding_busca = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  distancias = np.dot(np.stack(df["Embeddings"]), embedding_busca)

  indice = np.argmax(distancias)
  return df.iloc[indice]["Candidato"] + "\n" + df.iloc[indice]["Curriculo"]

<h2>Teste 1</h2>
Anúncio real para uma vaga intitulada "Engenheiro de IA Generativa Sr".<br>
O resultado retornou o currículo do Cientista de Dados.

In [16]:
consulta = "Qual currículo melhor atende aos seguintes requisitos:\n"
consulta += "Ter pelo menos 5 anos de experiência na área.\n"
consulta += "Ter postura proativa, observadora, organizada e comunicativa (atuação direta com cliente).\n"
consulta += "Habilidade de conduzir o planejamento de atividades do projeto conectando com demais integrantes da equipe.\n"
consulta += "Conhecimentos técnicos necessário: Experiência com IAs generativas / LLMs.\n"
consulta += "Preferencialmente com AWS Bedrock Experiência em Cloud AWS.\n"
consulta += "Sagemaker, lambda container, glue, terraform, bedrock, EC2, etc.\n"
consulta += "Desejável: Experiência no pipeline de produção de modelos ML.\n"
consulta += "Containerização de modelos / API.\n"
consulta += "Perfil consultor, entender os requisitos explícitos e implícitos, propondo soluções técnica"


curriculo_escolhido = buscar_melhor_candidato(consulta, df, model)
print(curriculo_escolhido)

Candidato 2
Cargo: Cientista de Dados
Resumo: Cientista de Dados com +3 anos de experiência em análise e modelagem de dados, com foco em Machine Learning e Inteligência Artificial. Habilidade comprovada em transformar dados em insights estratégicos para otimizar processos e auxiliar na tomada de decisões.
Habilidades: Python, R, Pandas, NumPy, Scikit-learn, TensorFlow, Keras, Machine Learning, Regressão, Classificação, Clusterização, Deep Learning, Tableau, Power BI, SQL Server, MySQL, Hadoop, Spark
Experiência Profissional: Cientista de Dados | Empresa A | [Cidade, Estado] | [Data Inicial] - [Data Final]
Analista de Dados | Empresa B | [Cidade, Estado] | [Data Inicial] - [Data Final]
Formação: Mestrado em Estatística | Universidade W | [Ano de Conclusão]
Bacharelado em Matemática | Universidade X | [Ano de Conclusão]
Certificações: AWS Certified Machine Learning - Specialty
Projetos: Desenvolvimento de um projeto de machine learning para [descrição do projeto], disponível no Github.
I

<h2>Teste 2</h2>
Anúncio real para uma vaga intitulada "Analista Programador / Desenvolvedor".<br>
O resultado retornou o currículo do Desenvolvedor Full Stack.

In [17]:
consulta = "Qual currículo melhor atende aos seguintes requisitos:\n"
consulta += "Desenvolvedor em linguagem C# com e NET ASP.NET. Versionador GitHub – Experiencia como FULL STACK: C#, ASP.NET, CSS, HTML, MVC, Webforms, padrão em camadas (DAL, BLL, MODEL) Jquery, Javascripty,Json,Bootstrapp. SQL SERVER, MYSQL, ORACLE. "

curriculo_escolhido = buscar_melhor_candidato(consulta, df, model)
print(curriculo_escolhido)

Candidato 1
Cargo: Desenvolvedor Full Stack
Resumo: Desenvolvedor Full Stack com +5 anos de experiência na construção e manutenção de aplicações web complexas, com foco em performance, escalabilidade e usabilidade. Habilidade comprovada em trabalhar com times ágeis, liderando projetos e entregando soluções inovadoras.
Habilidades: JavaScript, Python, Java, C#, React, Node.js, Angular, Django, Spring Boot, SQL Server, PostgreSQL, MongoDB, AWS, Azure, GCP, Git, Github, Scrum, Kanban
Experiência Profissional: Desenvolvedor Full Stack | Empresa X | [Cidade, Estado] | [Data Inicial] - [Data Final]
Desenvolvedor Front-End | Empresa Y | [Cidade, Estado] | [Data Inicial] - [Data Final]
Formação: Bacharelado em Ciência da Computação | Universidade Z | [Ano de Conclusão]
Certificações: 
Projetos: Desenvolvimento de um aplicativo mobile para [descrição do projeto], disponível na App Store e Google Play.
Criação de um blog pessoal sobre desenvolvimento web, compartilhando conhecimento e experiênci

<h2>Teste 3</h2>
Anúncio real para uma vaga intitulada "Gerente de Projetos - Sênior".<br>
O resultado retornou o currículo do Gerente de Projetos de TI.

In [18]:
consulta = "Qual currículo melhor atende aos seguintes requisitos:\n"
consulta += "Sólida experiência em gestão de projetos no segmento de Tecnologia.\n"
consulta += "Gerenciamento de equipe, demanda do projeto, elaboração do detalhamento do projeto e escopo.\n"
consulta += "Metodologia de gestão de projetos (PMBOK).\n"
consulta += "Elaboração de status reports dos projetos.\n"
consulta += "Conhecimento em metodologias ágeis.\n"
consulta += "Imprescindível possuir a certificação:  PMP.\n"
consulta += "Formação acadêmica: Graduação concluída na área de TI ou áreas afins.	"

curriculo_escolhido = buscar_melhor_candidato(consulta, df, model)
print(curriculo_escolhido)

Candidato 5
Cargo: Gerente de Projetos de TI
Resumo: Gerente de Projetos de TI com +7 anos de experiência em planejamento, execução e entrega de projetos de tecnologia. Habilidade comprovada em liderar equipes, gerenciar orçamentos, mitigar riscos e garantir a satisfação do cliente.
Habilidades: Waterfall, Agile (Scrum, Kanban), Jira, Microsoft Project, Asana, Gerenciamento de Riscos e Problemas, Comunicação Eficaz, Liderança e Motivação de Equipes, Negociação e Gestão de Conflitos
Experiência Profissional: Gerente de Projetos de TI | Empresa G | [Cidade, Estado] | [Data Inicial] - [Data Final]
Coordenador de Projetos de TI | Empresa H | [Cidade, Estado] | [Data Inicial] - [Data Final]
Formação: MBA em Gerenciamento de Projetos | Universidade W | [Ano de Conclusão]
Bacharelado em Ciência da Computação | Universidade X | [Ano de Conclusão]
Certificações: Project Management Professional (PMP)
Certified Scrum Master (CSM)
Projetos: Voluntariado em projetos de tecnologia para organizações 

<h1>Conclusão</h1>
O método de análise de currículos se mostrou muito eficiente para apontar o melhor currículo para cada anúncio testado.</br>
Em um cenário real com uma base de currículos maior, será interessante verificar os resultados de currículos com perfis semelhantes concorrendo para uma mesma vaga.